# This Demo shows up how to get started with Binance Bots
# By a breakdown of bot.py and its functions to understand
# step-by-step and how to own this software to do what you want


# /////////////////////////////////////////////////////////////////////////////////////////////////////////////



# Esta demo es una demonstración de como usar Bots en Binance
# A partir de explicar el paso a paso de bot.py y sus funciones
# para así entender el programa y empezar a modificarlo en base
# a lo que uno quiere que haga

# ( 1 ) Set up your API keys at binance | Genera tus Keys de Binance

## url : https://www.binance.com/en/support/faq/360002502072


# Format of api.py (file that has your keys
# Formato del api.py (archivo que tiene tus llaves)



# ( 2 ) Install all Packages / Instala todas las librerias



# pip install -r requirements.txt

In [1]:
!pip install -r requirements.txt

# The Bot Logic | Logica del Bot

# Idea: have a follow-trend system that is long when bullish and shorts when
# it is bearish (according to the market status). It has a stop loss, so it trigger  the condition or market trends reverse, so it reverse your positions

# -----------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------
# Idea: seguir la tendencia del mercado, si el mercado esta optimista mantenerse comprado caso contrario shortear (apostar a la baja)
# seguiremos la tendencia con un stop loss de garante, salvo que se active o se revierta la tendencia mantendremos posiciones, caso contrario revertimos



# Bot

# The block under is for showing purposes only, if you run it then stop it, it is designed to run forever
# el bloque siguiente es solo para demonstración, si lo corres despues paralo, esta diseñado para correrlo para siempre

In [2]:
# libraries to use \ librerias que usa

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd, yfinance as yahoo, os
import numpy as np, datetime as dt
import matplotlib.pyplot as plt
from pylab import mpl

# general endpoints with account info \ llamadas con datos de la cuenta
from controlpanel import *


from api import *
mpl.rcParams['font.family'] = 'serif'
plt.style.use('fivethirtyeight')
pd.options.display.width = 0
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows


# Login
client = Client(API_KEY,API_SECRET)

# alarm
duration, freq = 1, 440

# functions to buy if the bot is not running
# funciones para activar el bot si no esta prendido
def repo():
    direction = market()[0]
    for i in range(len(direction)):
        client.order_market_buy(
        symbol=str(direction.symbol.values[i]),
        quoteOrderQty=10.0
        )

def are_you_invested():
  account = pd.DataFrame(client.get_account()['balances'])
  account[['free','locked']] = account[['free','locked']].astype(float)
  holding = account[account['free']!=0.00000000]
  if len(holding) > 2:
    print("Portfolio is active, now lets monitor its performance")
  else:
    print("Starting the engines. Get ready for crypto")
    repo()

are_you_invested()

# function tell me the holdings of my account (from controlpanel script)
# function para saber el estado de mi cuenta (del script controlpanel)
holding = account()[0]
print(holding)

# Set stop-loss as a global variable   
# Seteamos stop-loss como variable global
stop = holding[['symbol','firstPrice']]
stop = stop.T
stop.columns = stop.iloc[0,:].to_list()
stop = stop.drop('symbol')
stop *= 0.9


# check market direction
# revisamos tendencia de mercado
def trending():
    df = yahoo.download('BTC-USD',period='30d',interval='2m')['Adj Close'].fillna(method='ffill')
    df = pd.DataFrame(df.values,columns=['Bitcoin'],index=df.index)
    df['SMA'] = df['Bitcoin'].rolling(round(len(df)*.1),min_periods=1).mean()
    trend = 'Short' if df['Bitcoin'].tail(1).values < df['SMA'].tail(1).values else 'Long'
    return trend

trend = trending()


symbols = sorted(holding.symbol.to_list())
pairs = sorted(holding.symbol.to_list())

# Gather info of pairs to have history to track
# Solicitamos la historia de las monedas a monitorear
for i in range(len(symbols)):
    klines = client.get_historical_klines(symbols[i],\
                                      Client.KLINE_INTERVAL_1MINUTE,\
                                          "5 days ago UTC")
    close = [float(k[4]) for k in klines]
    timestamp = [int(k[0]) for k in klines]
    for j in range(len(timestamp)):
      timestamp[j] = dt.datetime.fromtimestamp(timestamp[j]/1000).strftime('%Y-%m-%d %H:%M:%S.%f')
    
    symbols[i] = pd.DataFrame(close,columns=[f'{symbols[i]}'],index=timestamp)
    
data = symbols[0].tail(7000)
for i in range(1,len(symbols)):
  data[f'{symbols[i].columns[0]}'] = symbols[i].tail(7000).values

# init \ inicia 
twm = ThreadedWebsocketManager(API_KEY, API_SECRET)
twm.start()

# nominal quantity of our holdings
# tenencia nominal de nuestras monedas
nominal = holding[~(holding.symbol.str.contains('USDT')) & ~(holding.symbol.str.contains('BNB'))]
nominal = dict(zip(holding.symbol.to_list(), holding.nominalNow.to_list()))
lifespan = data.columns.to_list()

# main function with stop loss incorporated
# función principal para correr el programa
def handle_socket_message(response):
        global data
        discarded = []
        lifespan = data.columns.to_list()
        spot = response['data']['s']
        close = float(response['data']['c'])      
        timestamp = dt.datetime.fromtimestamp(int(response['data']['E'])/1000).strftime('%Y-%m-%d %H:%M:%S.%f')
        msg = pd.DataFrame(close,columns=[spot],index=[timestamp])
        data = pd.concat([data,msg]).sort_index().fillna(method='ffill')
        sma = data.rolling(round(len(data)*0.25)).min()*1.01
        tail = data.tail(1)-sma.tail(1).values
        alarm = tail[tail<0].dropna(axis=1) #if trend == 'Short' else tail[tail<0].dropna(axis=1)
        alarm = alarm.columns.to_list()
        #to sell dict(zip(alarm.asset, alarm.free))
        print(tail)
        if len(alarm) > 0:
            for i in alarm:
                if i in lifespan:
                    lifespan.remove(i)
                    #i = i.replace('USDT','')
                    ammount = nominal[i]
                    price = data[i].tail(1).values[0]  
                    val = int(price * ammount)
                    try: 
                        client.order_market_sell(
                            symbol=i,
                            quoteOrderQty=val
                            )
                        os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))
                        print(f"\n\n\n\tSelling {i} at {price} total obtained {val}\n\n\n")
                    except:
                        client.order_market_sell(
                            symbol=i,
                            quoteOrderQty=val-1
                            )
                        os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))
                        print(f"\n\n\n\tSelling {i} at {price} total obtained {val}\n\n\n")
                else:
                    pass
                               
            if lifespan == 0:
                print('\n\n\n\tGAME OVER\n\n\n')
                twm.stop()
            


streams = [i.lower() + '@miniTicker' for i in pairs]
twm.start_multiplex_socket(callback=handle_socket_message, streams=streams)
twm.join()



Portfolio is active, now lets monitor its performance


/home/lorenzo/Binance Trading Kit/DEMO/controlpanel.py:93: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cash = holding[(holding.asset.str.contains('BNB'))].append(holding[(holding.asset.str.contains('USDT'))])
/home/lorenzo/Binance Trading Kit/DEMO/controlpanel.py:111: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  book = book.append(pd.DataFrame(orders[i][-1].values(),index=orders[i][-1].keys()).T[['symbol','origQty','cummulativeQuoteQty','updateTime','side']])
/home/lorenzo/Binance Trading Kit/DEMO/controlpanel.py:111: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  book = book.append(pd.DataFrame(orders[i][-1].values(),index=orders[i][-1].keys()).T[['symbol','origQty','cummulativeQuoteQty','updateTime','side']])
/home/lore

        symbol  nominalNow  lastPrice   value Posture  \
0  ADADOWNUSDT 12,768.1300     0.0039 49.2850   Short   
1  BTCDOWNUSDT  1,447.5700     0.0348 50.3031   Short   
2  TRXDOWNUSDT     41.8700     1.1920 49.9090   Short   
3  XRPDOWNUSDT 53,242.4600     0.0009 49.5634   Short   

                         time  invested  nominalStart  firstPrice     PnL  
0  2022-09-05 14:57:40.874000   50.0000   12,768.1300      0.0039 -0.0143  
1  2022-09-05 14:57:42.677000   49.9998    1,447.5700      0.0345  0.0061  
2  2022-09-05 14:57:41.478000   49.9928       41.8700      1.1940 -0.0017  
3  2022-09-05 14:57:42.168000   50.0000   53,242.4600      0.0009 -0.0087  
[*********************100%***********************]  1 of 1 completed


/tmp/ipykernel_134576/3479016143.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{symbols[i].columns[0]}'] = symbols[i].tail(7000).values
/tmp/ipykernel_134576/3479016143.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{symbols[i].columns[0]}'] = symbols[i].tail(7000).values
/tmp/ipykernel_134576/3479016143.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

                            ADADOWNUSDT  BTCDOWNUSDT  TRXDOWNUSDT  XRPDOWNUSDT
2022-09-05 18:38:14.327000       0.0001       0.0007       0.0305       0.0000


KeyboardInterrupt: 

# Run it in the terminal or python spyder
# Correrlo en la terminal o en el spyder the python

# What you see as output? 
# its the current price of your holding minus the SMA trend price
# so if your holdings are gaining, it will be a positive number
# otherwise would be negative and trigger the reversal of your positions



# Que ves como output?
# el precio actual de tus tenencias restado el valor de la tendencia
# si el numero que ves es positivo mantenes la moneda, caso contrario 
# se vende y pasas a otra tendencia

In [3]:
twm.stop()
# Control Panel (controlpanel.py)

In [4]:
from controlpanel import *

holding = account()
holding

/home/lorenzo/Binance Trading Kit/DEMO/controlpanel.py:93: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cash = holding[(holding.asset.str.contains('BNB'))].append(holding[(holding.asset.str.contains('USDT'))])
/home/lorenzo/Binance Trading Kit/DEMO/controlpanel.py:111: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  book = book.append(pd.DataFrame(orders[i][-1].values(),index=orders[i][-1].keys()).T[['symbol','origQty','cummulativeQuoteQty','updateTime','side']])
/home/lorenzo/Binance Trading Kit/DEMO/controlpanel.py:111: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  book = book.append(pd.DataFrame(orders[i][-1].values(),index=orders[i][-1].keys()).T[['symbol','origQty','cummulativeQuoteQty','updateTime','side']])
/home/lore

[        symbol  nominalNow  lastPrice   value Posture  \
 0  ADADOWNUSDT 12,768.1300     0.0039 49.1956   Short   
 1  BTCDOWNUSDT  1,447.5700     0.0347 50.1684   Short   
 2  TRXDOWNUSDT     41.8700     1.1920 49.9090   Short   
 3  XRPDOWNUSDT 53,242.4600     0.0009 49.5634   Short   
 
                          time  invested  nominalStart  firstPrice     PnL  
 0  2022-09-05 14:57:40.874000   50.0000   12,768.1300      0.0039 -0.0161  
 1  2022-09-05 14:57:42.677000   49.9998    1,447.5700      0.0345  0.0034  
 2  2022-09-05 14:57:41.478000   49.9928       41.8700      1.1940 -0.0017  
 3  2022-09-05 14:57:42.168000   50.0000   53,242.4600      0.0009 -0.0087  ,
           symbol      value
 4            BNB     0.0004
 260      BNBDOWN 1,112.8400
 11          USDT    26.4656
 0    ADADOWNUSDT    49.1956
 1    BTCDOWNUSDT    50.1684
 2    TRXDOWNUSDT    49.9090
 3    XRPDOWNUSDT    49.5634]

In [5]:
# holdings tenencias
holding[0]

,symbol,nominalNow,lastPrice,value,Posture,time,invested,nominalStart,firstPrice,PnL
0,ADADOWNUSDT,"12,768.1300",0.0039,49.1956,Short,2022-09-05 14:57:40.874000,50.0000,"12,768.1300",0.0039,-0.0161
1,BTCDOWNUSDT,"1,447.5700",0.0347,50.1684,Short,2022-09-05 14:57:42.677000,49.9998,"1,447.5700",0.0345,0.0034
2,TRXDOWNUSDT,41.8700,1.1920,49.9090,Short,2022-09-05 14:57:41.478000,49.9928,41.8700,1.1940,-0.0017
3,XRPDOWNUSDT,"53,242.4600",0.0009,49.5634,Short,2022-09-05 14:57:42.168000,50.0000,"53,242.4600",0.0009,-0.0087


In [6]:
# value cash valor de las tenencias
holding[1]

,symbol,value
4,BNB,0.0004
260,BNBDOWN,"1,112.8400"
11,USDT,26.4656
0,ADADOWNUSDT,49.1956
1,BTCDOWNUSDT,50.1684
2,TRXDOWNUSDT,49.9090
3,XRPDOWNUSDT,49.5634
